## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable, Any
import copy

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


---

<a id="is_dominated"></a>
## is_dominated

*The is_dominated function takes two columns or two rows to determine if one observation is dominated by the other. The function also takes the current player to determine if it should check tuple position 0 or 1. The function returns true if the observation is dominated and false otherwise.* **Used by**: [eliminate_dominated_strategy](#eliminate_dominated_strategy)

* **row_1** List[Tuple]: the first row or column being compared
* **row_2**  List[Tuple]: the second row or column being compared
* **player**  int: the current player (either 0 for player 1 or 1 for player 2)
* **weak**  bool: a boolean specifying if the algorithm should consider weakly dominated strategies

**returns** bool

In [3]:
def is_dominated(row_1: List[Tuple], row_2: List[Tuple], player: int, weak: bool) -> bool:
    if weak == True:
        # compare to the next row or column
        if all(val_1[player] <= val_2[player] for val_1, val_2 in zip(row_1, row_2)):
            return True
    else:
        # compare to the next row or column
        if all(val_1[player] < val_2[player] for val_1, val_2 in zip(row_1, row_2)):
            return True
    return False

In [4]:
# assertions/unit tests
row_1 = [(0, 5), (1, 2), (3, 4)]
row_2 = [(1, 2), (3, 4), (5, 6)]

# check for player 1
assert is_dominated(row_1, row_2, 0, False) == True
# check for player 2
assert is_dominated(row_1, row_2, 1, False) == False

# test weakly dominated scenario
row_2 = [(0, 5), (3, 4), (5, 6)]
assert is_dominated(row_1, row_2, 0, True) == True
assert is_dominated(row_1, row_2, 0, False) == False

<a id="eliminate_dominated_strategy"></a>
## eliminate_dominated_strategy

*The eliminate_dominated_strategy function takes the current player's strategies, which is either a list of all rows in the game for player one or a list of all columns in the game for player two. The function also takes the current game state and remaining indices after previous eliminations. The function updates the game state and remaining indices for one ply based on the values provided for the current player and the boolean specifying whether weakly dominated strategies should count. The function returns the new game state, remaining indices, and a boolean specifying if the game state changed for this turn.* **Used by**: [player_one](#player_one), [player_two](#player_two)

* **strategies** List[List[Tuple]]: either a list of all rows in the game for player one or a list of all columns in the game for player two
* **game**  List[List[Tuple]]: the current game state
* **indices**  List[List[Tuple]]: the remaining indices that have not been eliminated
* **player**  int: the current player (either 0 for player 1 or 1 for player 2)
* **weak**  bool: a boolean specifying if the algorithm should consider weakly dominated strategies

**returns** List[List[List[Tuple]], List[List[Tuple]], bool]

In [5]:
def eliminate_dominated_strategy(strategies: List[List[Tuple]], game: List[List[Tuple]], indices: List[List[int]], player: int, weak: bool) -> List[Any]:
    new_game, new_indices = copy.deepcopy(game), copy.deepcopy(indices)
    for indx_1, row_1 in enumerate(strategies):
        for indx_2, row_2 in enumerate(strategies):
            if row_1 != row_2:
                dominated = is_dominated(row_1, row_2, player, weak)
                # update game and indicies if dominated strategy found        
                if dominated == True:
                    if player == 0:
                        del new_game[indx_1]
                        del new_indices[indx_1]
                    else:
                        new_game = [row[:indx_1] + row[indx_1 + 1:] for row in game]
                        new_indices = [row[:indx_1] + row[indx_1 + 1:] for row in indices]
                    return new_game, new_indices, True               
    return new_game, indices, False  

In [6]:
# assertions/unit tests
game = [[(10, 10), (14, 12)], [(12, 14), (20, 20)]]
# set payoffs as player two payoff (columns)
strategies = [[(10, 10), (12, 14)], [(14, 12), (20, 20)]]
indices = [[(0, 0), (0, 1)], [(1, 0), (1, 1)]]

# test with player 2
new_game, new_indices, state_changed = eliminate_dominated_strategy(strategies, game, indices, 1, False)
assert new_game == [[(14, 12)], [(20, 20)]]
assert new_indices == [[(0, 1)], [(1, 1)]]
assert state_changed == True

# test with player 1
strategies = game
new_game, new_indices, state_change = eliminate_dominated_strategy(strategies, game, indices, 0, False)
assert new_game == [[(12, 14), (20, 20)]]
assert new_indices == [[(1, 0), (1, 1)]]
assert state_changed == True

# test when state does not change
invalid_game = [[(0, 0)], [(0, 0)]]
strategies = invalid_game
invalid_indices = [[(0, 0)], [(1, 0)]]
new_game, new_indices, state_changed = eliminate_dominated_strategy(strategies, invalid_game, invalid_indices, 0, False)
assert new_game == invalid_game
assert new_indices == invalid_indices
assert state_changed == False

<a id="player_one"></a>
## player_one

*The player_one function reviews all game rows to determine if any dominated rows can be eliminated. The player_one function recursively calls the player_two function until reaching a terminal state. The player_one and player_two functions update the game state and indices array for every move. A terminal state is reached when the functions find a solution or when the game's state has not changed for 2 plies (indicating that neither player has an available move). The function returns the index of the pure strategy Nash Equilibrium or None if no equilibrium is found.* **Used by**: [solve_game](#solve_game), [player_two](player_two)

* **game** List[List[Tuple]]]: the current game state (positions that have not been eliminated)
* **indices**  List[List[Tuple]]: the remaining indices that have not been eliminated
* **eliminated**  bool: a boolean specifying if a value was eliminated by the opposite player on the previous move
* **weak**  bool: a boolean specifying if the algorithm should consider weakly dominated strategies

**returns** Tuple | None

In [7]:
def player_one(game: List[List[Tuple]], indices: List[List[Tuple]], eliminated: bool, weak: bool) -> Tuple | None:
    # break condition if we find a dominant strategy
    if len(game) == 1 and len(game[0]) == 1:
        return indices[0][0]
        
    new_game, new_indices, strategy_eliminated = eliminate_dominated_strategy(game, game, indices, 0, weak)

    if strategy_eliminated == False and eliminated == False:
        # the algorithm failed and there are no more dominant strategies
        return None
            
    return player_two(new_game, new_indices, strategy_eliminated, weak)

In [8]:
# assertions/unit tests
# See assertions for both player_one and player_two below. 
# The player functions call eachother recursively and cannot be tested until both functions are defined.

<a id="player_two"></a>
## player_two

*The player_two function reviews all game columns to determine if any dominated columns can be eliminated. The player_two function recursively calls the player_one function until reaching a terminal state. The player_one and player_two functions update the game state and indices array for every move. A terminal state is reached when the functions find a solution or when the game's state has not changed for 2 plies (indicating that neither player has an available move). The function returns the index of the pure strategy Nash Equilibrium or None if no equilibrium is found.* **Used by**: [solve_game](#solve_game), [player_one](player_one)

* **game** List[List[Tuple]]]: the current game state (positions that have not been eliminated)
* **indices**  List[List[Tuple]]: the remaining indices that have not been eliminated
* **eliminated**  bool: a boolean specifying if a value was eliminated by the opposite player on the previous move
* **weak**  bool: a boolean specifying if the algorithm should consider weakly dominated strategies

**returns** Tuple | None

In [9]:
def player_two(game: List[List[Tuple]], indices: List[List[Tuple]], eliminated: bool, weak: bool)-> Tuple | None:
    # break condition if we find an equilibrium
    if len(game) == 1 and len(game[0]) == 1:
        return indices[0][0]
        
    columns = [[row[i] for row in game] for i in range(len(game[0]))]
    new_game, new_indices, strategy_eliminated = eliminate_dominated_strategy(columns, game, indices, 1, weak)

    if strategy_eliminated == False and eliminated == False:
        # the algorithm failed and there are no more dominant strategies
        return None
            
    return player_one(new_game, new_indices, strategy_eliminated, weak)

In [10]:
# assertions/unit tests player_one
game = [[(10, 10), (14, 12)], [(12, 14), (20, 20)]]
indices = [[(0, 0), (0, 1)], [(1, 0), (1, 1)]]

# success case strongly dominated
assert player_one(game, indices, True, False) == (1, 1)

# success case weakly dominated
game = [[(10, 10), (14, 10)], [(12, 10), (14, 10)]]
assert player_one(game, indices, True, True) == (1, 1)
assert player_one(game, indices, True, False) == None

# failure case
game = [[(10, 10), (14, 9)], [(9, 13), (20, 20)]]
assert player_two(game, indices, True, False) == None

In [11]:
# assertions/unit tests player_two
game = [[(10, 10), (14, 12)], [(12, 14), (20, 20)]]
indices = [[(0, 0), (0, 1)], [(1, 0), (1, 1)]]

# success case strongly dominated
assert player_two(game, indices, True, False) == (1, 1)

# success case weakly dominated
game = [[(10, 10), (14, 10)], [(12, 10), (14, 20)]]
assert player_two(game, indices, True, True) == (1, 1)
assert player_two(game, indices, True, False) == None

# failure case
game = [[(10, 10), (14, 9)], [(9, 13), (20, 20)]]
assert player_two(game, indices, True, False) == None

<a id="solve_game"></a>
## solve_game

*The solve_game function applies the successive elimination of dominated strategies algorithm to find a pure strategy Nash Equilibrium. A Nash Equilibrium occurs when neither player can obtain a better outcome by changing their strategy alone. The algorithm considers strongly dominated strategies and weakly dominated strategies based on the value provided for the weak parameter. A strongly dominated strategy is one where all payoffs in strategy a for the current player are less than all payoffs in strategy b for the current player. In this scenario, strategy b strongly dominates strategy a, and the algorithm eliminates strategy a. A weakly dominated strategy is one where all payoffs in strategy a for the current player are less than or equal to all payoffs in strategy b for the current player. In this scenario, strategy b weakly dominates strategy a, and the algorithm eliminates strategy a only if weak is set to true. Player one eliminates strategies by row, and player two eliminates strategies by column. The algorithm alternates between player one and player two until finding an equilibrium or failing. The algorithm fails when neither player has a strategy to eliminate, and there is more than one remaining strategy in the game state. The algorithm returns the index of the pure strategy Nash Equilibrium if one is found.*

* **game**  List[List[Tuple]]: the game state of the form [[(10, 10), (14, 12), (14, 15)], [(12, 14), (20, 20), (28, 15)] where the first value in the tuples represents the payoffs for player one and the second value in the tuples represents the payoffs for player two. The rows represent the strategies for player one, and the columns represent the strategies for player two. 
* **weak**  bool: a boolean specifying if the algorithm should consider weakly dominated strategies

**returns** Tuple

In [12]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> Tuple:
    # track indices removed to return the final answer
    all_indices = [[(j, i) for i in range(len(game[0]))] for j in range(len(game))]
    equilibrium = player_two(game, all_indices, True, weak)

    # try starting with player one if no equilibrium found
    if equilibrium == None:
        equilibrium = player_one(game, all_indices, True, weak)
    return equilibrium

In [13]:
# assertions/unit tests
test_data_1 = [[(10, 10), (14, 12), (14, 15)], [(12, 14), (20, 20), (28, 15)], [(15, 14), (15, 28), (25, 25)]]
assert solve_game(test_data_1) == (1, 1)
assert solve_game(test_data_1, True) == (1, 1)

test_data_2 = [[(10, 10), (14, 12), (14, 15)], [(12, 14), (10, 10), (28, 15)], [(15, 14), (15, 28), (25, 25)]]
assert solve_game(test_data_2) == None
assert solve_game(test_data_2, True) == None

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (5, 5) | (10, 9) | (14, 13) |
|1  | (13, 14) | (10, 15) | (0, 0) |
|2  | (9, 10) | (11, 11) | (15, 10) |


**Algorithm steps starting with player two:**
- Player 2 eliminates column 0 because it is strongly dominated by column 1
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(5, 5)~~ | (10, 9) | (14, 13) |
|1  | ~~(13, 14)~~ | (10, 15) | (0, 0) |
|2  | ~~(9, 10)~~ | (11, 11) | (15, 10) |
- Player 1 eliminates row 0 because it is strongly dominated by row 2
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(5, 5)~~ | ~~(10, 9)~~ | ~~(14, 13)~~ |
|1  | ~~(13, 14)~~ | (10, 15) | (0, 0) |
|2  | ~~(9, 10)~~ | (11, 11) | (15, 10) |
-  Player 2 eliminates column 2 because it is strongly dominated by column 1
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(5, 5)~~ | ~~(10, 9)~~ | ~~(14, 13)~~ |
|1  | ~~(13, 14)~~ | (10, 15) | ~~(0, 0)~~ |
|2  | ~~(9, 10)~~ | (11, 11) | ~~(15, 10)~~ |
-  Player 1 eliminates row 1 because it is strongly dominated by row 2
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(5, 5)~~ | ~~(10, 9)~~ | ~~(14, 13)~~ |
|1  | ~~(13, 14)~~ | ~~(10, 15)~~ | ~~(0, 0)~~ |
|2  | ~~(9, 10)~~ | (11, 11) | ~~(15, 10)~~ |

-  Solution reached at (2, 1)

**Solution:** (2, 1)

In [14]:
test_game_1 = [[(5, 5), (10, 9), (14, 13)], [(13, 14), (10, 15), (0, 0)], [(9, 10), (11, 11), (15, 10)]]

solution = solve_game(test_game_1)

print(f"Solution: {solution}")

Solution: (2, 1)


In [15]:
assert solution == (2, 1)

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (9, 10) | (10, 11) | (15, 10) |
|1  | (13, 14) | (10, 15) | (0, 0) |
|2  | (5, 5) | (10, 9) | (14, 13) |


**Algorithm steps starting with player two:**

- Player 2 eliminates column 0 because it is strongly dominated by column 1
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(9, 10)~~ | (10, 11) | (15, 10) |
|1  | ~~(13, 14)~~ | (10, 15) | (0, 0) |
|2  | ~~(5, 5)~~ | (10, 9) | (14, 13) |
- Player 1 eliminates row 1 because it is weakly dominated by row 0
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(9, 10)~~ | (10, 11) | (15, 10) |
|1  | ~~(13, 14)~~ | ~~(10, 15)~~ | ~~(0, 0)~~ |
|2  | ~~(5, 5)~~ | (10, 9) | (14, 13) |
- Player 2 has nothing to eliminate
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(9, 10)~~ | (10, 11) | (15, 10) |
|1  | ~~(13, 14)~~ | ~~(10, 15)~~ | ~~(0, 0)~~ |
|2  | ~~(5, 5)~~ | (10, 9) | (14, 13) |
- Player 1 eliminates row 2 because it is weakly dominated by row 0
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(9, 10)~~ | (10, 11) | (15, 10) |
|1  | ~~(13, 14)~~ | ~~(10, 15)~~ | ~~(0, 0)~~ |
|2  | ~~(5, 5)~~ | ~~(10, 9)~~ | ~~(14, 13)~~ |
- Player 2 eliminates column 2 because it is strongly dominated by column 1
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~(9, 10)~~ | (10, 11) | ~~(15, 10)~~ |
|1  | ~~(13, 14)~~ | ~~(10, 15)~~ | ~~(0, 0)~~ |
|2  | ~~(5, 5)~~ | ~~(10, 9)~~ | ~~(14, 13)~~ |
- Solution reached at (0, 1)

**Solution:** (0, 1)

In [16]:
test_game_2 = [[(9, 10), (10, 11), (15, 10)], [(13, 14), (10, 15), (0, 0)], [(5, 5), (10, 9), (14, 13)]]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)
print(f"Weak Solution: {weak_solution}")

Weak Solution: (0, 1)


In [17]:
assert strong_solution == None
assert weak_solution == (0, 1)

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (5, 5) | (10, 9) | (14, 13) |
|1  | (9, 10) | (1, 1) | (23, 25) |
|2  | (13, 14) | (25, 23) | (0, 0) |

**Algorithm steps starting with player two:**

- Player 2 cannot eliminate anything
- Player 1 cannot eliminate anything
- Algorithm fails and returns None

  
**Solution:** None

In [18]:
test_game_3 = [[(5, 5), (10, 9), (14, 13)], [(9, 10), (1, 1), (23, 25)], [(13, 14), (25, 23), (0, 0)]]

strong_solution = solve_game(test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [19]:
assert strong_solution == None
assert weak_solution == None